# Projeto Turing - engenharia de colunas(atríbutos)  
*Metodologia:*  
Transformar colunas de nossa base de dados em informação, e trazer também novas informações,  
enriquecendo assim o nosso modelo, por final espera-se um data-frame único e mais robusto   

In [27]:
import pandas as pd

In [28]:
df = pd.read_csv(r"C:\Users\Kaue Mandarino\Desktop\Mestrado\degrees\Projeto-Turin\Dados\2 - Dados apos a limpeza e de para.csv", encoding="latin-1")
df.head(3)

,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,ESTCIV,ESC,...,ESCFALAGR1,TPNIVELINV,CAUSABAS_O,TPPOS,LINHAA_DESC,LINHAB_DESC,LINHAC_DESC,LINHAD_DESC,LINHAII_DESC,CAUSABAS_DESC
0,2023-02-14,20:24:00,Rio Grande do Sul,Porto Alegre,1981-02-26,41.0,Feminino,Branca,Solteiro,Ignorado,...,Ignorado,M,X700,NÃ£o informado,NÃ£o informado,NÃ£o informado,NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang...",NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang..."
1,2023-01-01,NaN,Santa Catarina,FlorianÃ³polis,1982-05-09,40.0,Feminino,Branca,Solteiro,12 anos ou mais,...,Ensino Superior completo,M,X804,NÃ£o informado,Traumatismos mÃºltiplos nÃ£o especificados,LesÃ£o autoprovocada por salto,NÃ£o informado,NÃ£o informado,NÃ£o informado,LesÃ£o autoprovocada por salto
2,2023-01-01,NaN,Estrangeiro,NaN,1990-11-29,32.0,Feminino,Branca,Solteiro,4 a 7 anos,...,Fundamental II incompleto ou inespecÃ­fico,M,X689,NÃ£o informado,NÃ£o informado,NÃ£o informado,Efeito tÃ³xico de pesticidas,AutointoxicaÃ§Ã£o por pesticidas,NÃ£o informado,AutointoxicaÃ§Ã£o por pesticidas


In [29]:
# Tenho uma tabela em excel que trouxe informações complementares nomeadas como intervalo, vamos usar a bibliotca openpyxl para ver quais são:
from openpyxl import load_workbook

arquivo = r"C:\Users\Kaue Mandarino\Desktop\Mestrado\degrees\Projeto-Turin\Dados\Modelo ERD e Tabelas de Apoio.xlsx"

wb = load_workbook(arquivo, data_only=True)

# Lista todos os nomes definidos
nomes = wb.defined_names.definedName

for nome in nomes:
    print(f"Nome: {nome.name}")
    print(f"Refere-se a: {nome.attr_text}")
    print("-" * 40)



Nome: dim_faixa_etaria
Refere-se a: outras_tabelas!$A$90:$D$100
----------------------------------------
Nome: dim_regioes
Refere-se a: outras_tabelas!$A$22:$C$49
----------------------------------------
Nome: f_regioes
Refere-se a: outras_tabelas!$A$52:$F$57
----------------------------------------
Nome: faixa_horaria
Refere-se a: outras_tabelas!$A$10:$E$19
----------------------------------------
Nome: tab_b_desc
Refere-se a: outras_tabelas!$A$129:$C$162
----------------------------------------
Nome: tb_b_desc
Refere-se a: outras_tabelas!$A$130:$C$162
----------------------------------------
Nome: tb_desc_a
Refere-se a: outras_tabelas!$A$103:$C$127
----------------------------------------
Nome: tb_estacoes
Refere-se a: outras_tabelas!$A$2:$D$7
----------------------------------------


In [30]:
# Agora vamos trazer estas tabelas para nosso modelo:

from openpyxl.utils import range_boundaries

arquivo = r"C:\Users\Kaue Mandarino\Desktop\Mestrado\degrees\Projeto-Turin\Dados\Modelo ERD e Tabelas de Apoio.xlsx"
aba = "outras_tabelas"

wb = load_workbook(arquivo, data_only=True)
ws = wb[aba]


def ler_intervalo(ws, intervalo):
    min_col, min_row, max_col, max_row = range_boundaries(intervalo)

    dados = list(
        ws.iter_rows(
            min_row=min_row,
            max_row=max_row,
            min_col=min_col,
            max_col=max_col,
            values_only=True
        )
    )

    return pd.DataFrame(dados[1:], columns=dados[0])


# =========================
# DIMENSÕES / TABELAS
# =========================

dim_faixa_etaria = ler_intervalo(ws, "A90:D100")

dim_regioes = ler_intervalo(ws, "A22:B49")

f_regioes = ler_intervalo(ws, "A52:F57")

faixa_horaria = ler_intervalo(ws, "A10:E19")

tab_b_desc = ler_intervalo(ws, "A129:C162")

tb_b_desc = ler_intervalo(ws, "A130:C162")

tb_desc_a = ler_intervalo(ws, "A103:C127")

tb_estacoes = ler_intervalo(ws, "A2:D7")


Por fim vamos começar a engenharia de colunas de fato

In [31]:
df.head(3)

,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,ESTCIV,ESC,...,ESCFALAGR1,TPNIVELINV,CAUSABAS_O,TPPOS,LINHAA_DESC,LINHAB_DESC,LINHAC_DESC,LINHAD_DESC,LINHAII_DESC,CAUSABAS_DESC
0,2023-02-14,20:24:00,Rio Grande do Sul,Porto Alegre,1981-02-26,41.0,Feminino,Branca,Solteiro,Ignorado,...,Ignorado,M,X700,NÃ£o informado,NÃ£o informado,NÃ£o informado,NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang...",NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang..."
1,2023-01-01,NaN,Santa Catarina,FlorianÃ³polis,1982-05-09,40.0,Feminino,Branca,Solteiro,12 anos ou mais,...,Ensino Superior completo,M,X804,NÃ£o informado,Traumatismos mÃºltiplos nÃ£o especificados,LesÃ£o autoprovocada por salto,NÃ£o informado,NÃ£o informado,NÃ£o informado,LesÃ£o autoprovocada por salto
2,2023-01-01,NaN,Estrangeiro,NaN,1990-11-29,32.0,Feminino,Branca,Solteiro,4 a 7 anos,...,Fundamental II incompleto ou inespecÃ­fico,M,X689,NÃ£o informado,NÃ£o informado,NÃ£o informado,Efeito tÃ³xico de pesticidas,AutointoxicaÃ§Ã£o por pesticidas,NÃ£o informado,AutointoxicaÃ§Ã£o por pesticidas


In [32]:
dim_faixa_etaria.head(3)

,ordem_faixa,faixa_etaria,idade_inicio,idade_fim
0,1,Primeira Infância,0,5
1,2,Infância,6,11
2,3,Adolescência,12,17


In [33]:
dim_calendario = pd.read_excel(r"C:\Users\Kaue Mandarino\Desktop\Mestrado\degrees\Projeto-Turin\Dados\Modelo ERD e Tabelas de Apoio.xlsx", sheet_name="dim_calendario")
dim_calendario.head(3)

,Data,Dia da Semana,Num Dia Semana,Final de Semana,Feriado Nacional,Nome do Feriado,Dia do Ano,Semana do Ano,Mês
0,2023-01-01,domingo,7,Sim,Sim,Confraternização Universal,1,1,1
1,2023-01-02,segunda-feira,1,Não,Não,NaN,2,2,1
2,2023-01-03,terça-feira,2,Não,Não,NaN,3,2,1


Uma vez que importamos todas as tabelas que vamos trabalhar, para o PANDAR, vamos comseçar a uní-las!

In [34]:
dim_faixa_etaria

,ordem_faixa,faixa_etaria,idade_inicio,idade_fim
0,1,Primeira Infância,0,5
1,2,Infância,6,11
2,3,Adolescência,12,17
3,4,Juventude Inicial,18,24
4,5,Juventude Plena,25,34
5,6,Adulto Inicial,35,44
6,7,Adulto Pleno,45,59
7,8,Idoso Inicial,60,69
8,9,Idoso Pleno,70,79
9,10,Longevidade,80,120


In [35]:
df.head(3)

,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,ESTCIV,ESC,...,ESCFALAGR1,TPNIVELINV,CAUSABAS_O,TPPOS,LINHAA_DESC,LINHAB_DESC,LINHAC_DESC,LINHAD_DESC,LINHAII_DESC,CAUSABAS_DESC
0,2023-02-14,20:24:00,Rio Grande do Sul,Porto Alegre,1981-02-26,41.0,Feminino,Branca,Solteiro,Ignorado,...,Ignorado,M,X700,NÃ£o informado,NÃ£o informado,NÃ£o informado,NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang...",NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang..."
1,2023-01-01,NaN,Santa Catarina,FlorianÃ³polis,1982-05-09,40.0,Feminino,Branca,Solteiro,12 anos ou mais,...,Ensino Superior completo,M,X804,NÃ£o informado,Traumatismos mÃºltiplos nÃ£o especificados,LesÃ£o autoprovocada por salto,NÃ£o informado,NÃ£o informado,NÃ£o informado,LesÃ£o autoprovocada por salto
2,2023-01-01,NaN,Estrangeiro,NaN,1990-11-29,32.0,Feminino,Branca,Solteiro,4 a 7 anos,...,Fundamental II incompleto ou inespecÃ­fico,M,X689,NÃ£o informado,NÃ£o informado,NÃ£o informado,Efeito tÃ³xico de pesticidas,AutointoxicaÃ§Ã£o por pesticidas,NÃ£o informado,AutointoxicaÃ§Ã£o por pesticidas


## 🧱 Classificação Etária e Associação com Dimensão

Nesta etapa, a base principal é enriquecida com informações da dimensão
de faixa etária a partir da coluna `IDADE`.

### 🔹 Procedimento

- A coluna `IDADE` é padronizada para o tipo inteiro, mantendo valores ausentes.
- É criada a chave `ordem_faixa`, que classifica cada registro em uma faixa etária
  com base em intervalos predefinidos.
- A base é associada à dimensão `dim_faixa_etaria` utilizando essa chave,
  sem perda de registros.
- As colunas provenientes da dimensão recebem o prefixo `dim_faixa_etaria.`
  para manter clareza e governança do modelo analítico.

### 🎯 Resultado

Cada registro passa a conter sua classificação etária e os atributos
descritivos da dimensão correspondente, preservando integralmente a base
original para análises e visualizações.


In [36]:
# Garantir IDADE como inteiro
df["IDADE"] = df["IDADE"].astype("Int64")

# Criar a chave de faixa etária a partir da idade
df["ordem_faixa"] = pd.cut(
    df["IDADE"],
    bins=[-1, 5, 11, 17, 24, 34, 44, 59, 69, 79, 120],
    labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
).astype("Int64")

# Join com a dimensão pela chave (não perde linha)
df = df.merge(
    dim_faixa_etaria,
    on="ordem_faixa",
    how="left"
)

# Prefixar colunas da dimensão
prefixo = "dim_faixa_etaria."
novos_nomes = {}

for coluna in dim_faixa_etaria.columns:
    if coluna != "ordem_faixa":
        novos_nomes[coluna] = prefixo + coluna

df = df.rename(columns=novos_nomes)


In [37]:
df[["IDADE", "ordem_faixa"] + [c for c in df.columns if c.startswith("dim_faixa_etaria.")]]

,IDADE,ordem_faixa,dim_faixa_etaria.faixa_etaria,dim_faixa_etaria.idade_inicio,dim_faixa_etaria.idade_fim
0,41,6,Adulto Inicial,35.0,44.0
1,40,6,Adulto Inicial,35.0,44.0
2,32,5,Juventude Plena,25.0,34.0
3,60,8,Idoso Inicial,60.0,69.0
4,31,5,Juventude Plena,25.0,34.0
...,...,...,...,...,...
16254,53,7,Adulto Pleno,45.0,59.0
16255,18,4,Juventude Inicial,18.0,24.0
16256,12,3,Adolescência,12.0,17.0
16257,21,4,Juventude Inicial,18.0,24.0


In [38]:
df = df.drop(columns=["ordem_faixa"])

In [39]:
df

,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,ESTCIV,ESC,...,TPPOS,LINHAA_DESC,LINHAB_DESC,LINHAC_DESC,LINHAD_DESC,LINHAII_DESC,CAUSABAS_DESC,dim_faixa_etaria.faixa_etaria,dim_faixa_etaria.idade_inicio,dim_faixa_etaria.idade_fim
0,2023-02-14,20:24:00,Rio Grande do Sul,Porto Alegre,1981-02-26,41,Feminino,Branca,Solteiro,Ignorado,...,NÃ£o informado,NÃ£o informado,NÃ£o informado,NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang...",NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang...",Adulto Inicial,35.0,44.0
1,2023-01-01,NaN,Santa Catarina,FlorianÃ³polis,1982-05-09,40,Feminino,Branca,Solteiro,12 anos ou mais,...,NÃ£o informado,Traumatismos mÃºltiplos nÃ£o especificados,LesÃ£o autoprovocada por salto,NÃ£o informado,NÃ£o informado,NÃ£o informado,LesÃ£o autoprovocada por salto,Adulto Inicial,35.0,44.0
2,2023-01-01,NaN,Estrangeiro,NaN,1990-11-29,32,Feminino,Branca,Solteiro,4 a 7 anos,...,NÃ£o informado,NÃ£o informado,NÃ£o informado,Efeito tÃ³xico de pesticidas,AutointoxicaÃ§Ã£o por pesticidas,NÃ£o informado,AutointoxicaÃ§Ã£o por pesticidas,Juventude Plena,25.0,34.0
3,2023-01-01,NaN,Rio Grande do Sul,Sobradinho,1962-12-07,60,Masculino,Branca,Solteiro,4 a 7 anos,...,NÃ£o informado,LesÃ£o autoprovocada por salto,NÃ£o informado,NÃ£o informado,NÃ£o informado,NÃ£o informado,LesÃ£o autoprovocada por salto,Idoso Inicial,60.0,69.0
4,2023-01-01,NaN,Rio Grande do Sul,VenÃ¢ncio Aires,1991-01-15,31,Masculino,Branca,Solteiro,Ignorado,...,NÃ£o informado,Asfixia,NÃ£o informado,NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang...",NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang...",Juventude Plena,25.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16254,2023-11-18,NaN,Minas Gerais,Santo AntÃ´nio do Rio Abaixo,1970-08-15,53,Masculino,Parda,Ignorado,Nenhuma,...,NÃ£o informado,Asfixia,"LesÃ£o autoprovocada por enforcamento, estrang...",NÃ£o informado,NÃ£o informado,NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang...",Adulto Pleno,45.0,59.0
16255,2023-09-27,20:00:00,AmapÃ¡,Pedra Branca do Amapari,2005-03-26,18,Masculino,IndÃ­gena,Ignorado,8 a 11 anos,...,NÃ£o informado,Asfixia,"LesÃ£o autoprovocada por enforcamento, estrang...",NÃ£o informado,NÃ£o informado,NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang...",Juventude Inicial,18.0,24.0
16256,2023-11-01,19:00:00,Amazonas,EirunepÃ©,2011-01-01,12,Feminino,IndÃ­gena,Ignorado,Ignorado,...,NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang...",NÃ£o informado,NÃ£o informado,NÃ£o informado,NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang...",Adolescência,12.0,17.0
16257,2023-11-26,15:40:00,Amazonas,EirunepÃ©,2002-01-01,21,Feminino,IndÃ­gena,Ignorado,Nenhuma,...,NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang...",NÃ£o informado,NÃ£o informado,NÃ£o informado,NÃ£o informado,"LesÃ£o autoprovocada por enforcamento, estrang...",Juventude Inicial,18.0,24.0


Agora vamos trabalhar na DTOBITO

In [40]:
df['DTOBITO'].head(3)

0    2023-02-14
1    2023-01-01
2    2023-01-01
Name: DTOBITO, dtype: object

In [41]:
dim_calendario.head(3)


,Data,Dia da Semana,Num Dia Semana,Final de Semana,Feriado Nacional,Nome do Feriado,Dia do Ano,Semana do Ano,Mês
0,2023-01-01,domingo,7,Sim,Sim,Confraternização Universal,1,1,1
1,2023-01-02,segunda-feira,1,Não,Não,NaN,2,2,1
2,2023-01-03,terça-feira,2,Não,Não,NaN,3,2,1


## 📅 Associação da Data do Óbito com a Dimensão Calendário

Nesta etapa, a base principal é enriquecida com atributos temporais
por meio da associação da data do óbito (`DTOBITO`) com a dimensão
calendário (`dim_calendario`).

### 🔹 Procedimento

- A coluna `DTOBITO` da base principal e a coluna `Data` da dimensão
  calendário são padronizadas para o tipo data.
- É realizado um relacionamento do tipo *left join*, garantindo que
  nenhum registro da base principal seja perdido.
- Para cada data de óbito, são incorporados atributos temporais como
  dia da semana, feriado, semana do ano e mês.
- As colunas provenientes da dimensão calendário recebem o prefixo
  `dim_calendario.` para manter clareza, rastreabilidade e governança
  do modelo analítico.

### 🎯 Resultado

Cada registro da base principal passa a conter informações temporais
derivadas da dimensão calendário, possibilitando análises por período,
sazonalidade e comportamento ao longo do tempo, sem alteração da
granularidade original dos dados.


In [42]:
# Garantir que as datas estão no mesmo tipo
df["DTOBITO"] = pd.to_datetime(df["DTOBITO"], errors="coerce")
dim_calendario["Data"] = pd.to_datetime(dim_calendario["Data"], errors="coerce")

# Join com a dimensão calendário
df = df.merge(
    dim_calendario,
    left_on="DTOBITO",
    right_on="Data",
    how="left"
)

# Prefixar colunas da dimensão calendário
prefixo = "dim_calendario."
novos_nomes = {}

for coluna in dim_calendario.columns:
    novos_nomes[coluna] = prefixo + coluna

df = df.rename(columns=novos_nomes)


In [43]:
df[["DTOBITO"] + [c for c in df.columns if c.startswith("dim_calendario.")]]

,DTOBITO,dim_calendario.Data,dim_calendario.Dia da Semana,dim_calendario.Num Dia Semana,dim_calendario.Final de Semana,dim_calendario.Feriado Nacional,dim_calendario.Nome do Feriado,dim_calendario.Dia do Ano,dim_calendario.Semana do Ano,dim_calendario.Mês
0,2023-02-14,2023-02-14,terça-feira,2,Não,Não,NaN,45,8,2
1,2023-01-01,2023-01-01,domingo,7,Sim,Sim,Confraternização Universal,1,1,1
2,2023-01-01,2023-01-01,domingo,7,Sim,Sim,Confraternização Universal,1,1,1
3,2023-01-01,2023-01-01,domingo,7,Sim,Sim,Confraternização Universal,1,1,1
4,2023-01-01,2023-01-01,domingo,7,Sim,Sim,Confraternização Universal,1,1,1
...,...,...,...,...,...,...,...,...,...,...
16254,2023-11-18,2023-11-18,sábado,6,Sim,Não,NaN,322,47,11
16255,2023-09-27,2023-09-27,quarta-feira,3,Não,Não,NaN,270,40,9
16256,2023-11-01,2023-11-01,quarta-feira,3,Não,Não,NaN,305,45,11
16257,2023-11-26,2023-11-26,domingo,7,Sim,Não,NaN,330,48,11


In [44]:
df

,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,ESTCIV,ESC,...,dim_faixa_etaria.idade_fim,dim_calendario.Data,dim_calendario.Dia da Semana,dim_calendario.Num Dia Semana,dim_calendario.Final de Semana,dim_calendario.Feriado Nacional,dim_calendario.Nome do Feriado,dim_calendario.Dia do Ano,dim_calendario.Semana do Ano,dim_calendario.Mês
0,2023-02-14,20:24:00,Rio Grande do Sul,Porto Alegre,1981-02-26,41,Feminino,Branca,Solteiro,Ignorado,...,44.0,2023-02-14,terça-feira,2,Não,Não,NaN,45,8,2
1,2023-01-01,NaN,Santa Catarina,FlorianÃ³polis,1982-05-09,40,Feminino,Branca,Solteiro,12 anos ou mais,...,44.0,2023-01-01,domingo,7,Sim,Sim,Confraternização Universal,1,1,1
2,2023-01-01,NaN,Estrangeiro,NaN,1990-11-29,32,Feminino,Branca,Solteiro,4 a 7 anos,...,34.0,2023-01-01,domingo,7,Sim,Sim,Confraternização Universal,1,1,1
3,2023-01-01,NaN,Rio Grande do Sul,Sobradinho,1962-12-07,60,Masculino,Branca,Solteiro,4 a 7 anos,...,69.0,2023-01-01,domingo,7,Sim,Sim,Confraternização Universal,1,1,1
4,2023-01-01,NaN,Rio Grande do Sul,VenÃ¢ncio Aires,1991-01-15,31,Masculino,Branca,Solteiro,Ignorado,...,34.0,2023-01-01,domingo,7,Sim,Sim,Confraternização Universal,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16254,2023-11-18,NaN,Minas Gerais,Santo AntÃ´nio do Rio Abaixo,1970-08-15,53,Masculino,Parda,Ignorado,Nenhuma,...,59.0,2023-11-18,sábado,6,Sim,Não,NaN,322,47,11
16255,2023-09-27,20:00:00,AmapÃ¡,Pedra Branca do Amapari,2005-03-26,18,Masculino,IndÃ­gena,Ignorado,8 a 11 anos,...,24.0,2023-09-27,quarta-feira,3,Não,Não,NaN,270,40,9
16256,2023-11-01,19:00:00,Amazonas,EirunepÃ©,2011-01-01,12,Feminino,IndÃ­gena,Ignorado,Ignorado,...,17.0,2023-11-01,quarta-feira,3,Não,Não,NaN,305,45,11
16257,2023-11-26,15:40:00,Amazonas,EirunepÃ©,2002-01-01,21,Feminino,IndÃ­gena,Ignorado,Nenhuma,...,24.0,2023-11-26,domingo,7,Sim,Não,NaN,330,48,11


## 🌤️ Classificação Sazonal dos Registros (Dimensão Estações)

Nesta etapa, a base principal é enriquecida com informações sazonais
por meio da criação de uma chave âncora (`ordem_estacao`) derivada da
data do óbito (`DTOBITO`).

### 🔹 Procedimento

- A coluna `DTOBITO` é padronizada para o tipo data.
- A partir da data, é criada a chave `ordem_estacao`, que classifica
  cada registro em uma estação do ano, de acordo com períodos
  previamente definidos.
- A associação com a dimensão `tb_estacoes` é realizada por meio de um
  *left join*, garantindo que nenhum registro da base principal seja
  perdido.
- As colunas provenientes da dimensão recebem o prefixo
  `dim_estacoes.` para assegurar clareza, rastreabilidade e governança
  do modelo analítico.

### 🎯 Resultado

Cada registro passa a conter sua classificação sazonal e os atributos
descritivos da estação correspondente, mantendo a granularidade
original dos dados e possibilitando análises temporais e sazonais
consistentes.


In [45]:
# Garantir datas
df["DTOBITO"] = pd.to_datetime(df["DTOBITO"], errors="coerce")

# Criar chave âncora de estação a partir da data
df["ordem_estacao"] = pd.cut(
    df["DTOBITO"],
    bins=[
        pd.Timestamp("2022-12-20"),
        pd.Timestamp("2023-03-20"),
        pd.Timestamp("2023-06-20"),
        pd.Timestamp("2023-09-22"),
        pd.Timestamp("2023-12-20"),
        pd.Timestamp("2023-12-31")
    ],
    labels=[1, 2, 3, 4, 5]
).astype("Int64")

# Join simples pela chave (não perde linha)
df = df.merge(
    tb_estacoes,
    on="ordem_estacao",
    how="left"
)

# Prefixar colunas da dimensão estações
prefixo = "dim_estacoes."
novos_nomes = {}

for coluna in tb_estacoes.columns:
    if coluna != "ordem_estacao":
        novos_nomes[coluna] = prefixo + coluna

df = df.rename(columns=novos_nomes)
df[["DTOBITO"] + [c for c in df.columns if c.startswith("dim_calendario.") or c.startswith("dim_estacoes.")]]

,DTOBITO,dim_calendario.Data,dim_calendario.Dia da Semana,dim_calendario.Num Dia Semana,dim_calendario.Final de Semana,dim_calendario.Feriado Nacional,dim_calendario.Nome do Feriado,dim_calendario.Dia do Ano,dim_calendario.Semana do Ano,dim_calendario.Mês,dim_estacoes.estacao,dim_estacoes.inicio,dim_estacoes.fim
0,2023-02-14,2023-02-14,terça-feira,2,Não,Não,NaN,45,8,2,Verão,2022-12-21,2023-03-20
1,2023-01-01,2023-01-01,domingo,7,Sim,Sim,Confraternização Universal,1,1,1,Verão,2022-12-21,2023-03-20
2,2023-01-01,2023-01-01,domingo,7,Sim,Sim,Confraternização Universal,1,1,1,Verão,2022-12-21,2023-03-20
3,2023-01-01,2023-01-01,domingo,7,Sim,Sim,Confraternização Universal,1,1,1,Verão,2022-12-21,2023-03-20
4,2023-01-01,2023-01-01,domingo,7,Sim,Sim,Confraternização Universal,1,1,1,Verão,2022-12-21,2023-03-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16254,2023-11-18,2023-11-18,sábado,6,Sim,Não,NaN,322,47,11,Primavera,2023-09-23,2023-12-20
16255,2023-09-27,2023-09-27,quarta-feira,3,Não,Não,NaN,270,40,9,Primavera,2023-09-23,2023-12-20
16256,2023-11-01,2023-11-01,quarta-feira,3,Não,Não,NaN,305,45,11,Primavera,2023-09-23,2023-12-20
16257,2023-11-26,2023-11-26,domingo,7,Sim,Não,NaN,330,48,11,Primavera,2023-09-23,2023-12-20


In [46]:
# Garantir HORAOBITO como horário
df["HORAOBITO"] = pd.to_datetime(df["HORAOBITO"], errors="coerce").dt.time

# Criar chave âncora de subfaixa horária
df["ordem_subfaixa"] = pd.cut(
    pd.to_timedelta(
        pd.to_datetime(df["HORAOBITO"].astype(str), errors="coerce").dt.strftime("%H:%M:%S")
    ),
    bins=[
        pd.Timedelta("00:00:00"),
        pd.Timedelta("02:59:59"),
        pd.Timedelta("05:59:59"),
        pd.Timedelta("08:59:59"),
        pd.Timedelta("11:29:59"),
        pd.Timedelta("13:29:59"),
        pd.Timedelta("15:29:59"),
        pd.Timedelta("17:59:59"),
        pd.Timedelta("20:59:59"),
        pd.Timedelta("23:59:59")
    ],
    labels=[1, 2, 3, 4, 5, 6, 7, 8, 9],
    include_lowest=True
).astype("Int64")

# Join com a dimensão faixa horária
df = df.merge(
    faixa_horaria,
    on="ordem_subfaixa",
    how="left"
)

# Prefixar colunas da dimensão faixa horária
prefixo = "dim_faixa_horaria."
novos_nomes = {}

for coluna in faixa_horaria.columns:
    if coluna != "ordem_subfaixa":
        novos_nomes[coluna] = prefixo + coluna

df = df.rename(columns=novos_nomes)


C:\Users\Kaue Mandarino\AppData\Local\Temp\ipykernel_7500\2821733182.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["HORAOBITO"] = pd.to_datetime(df["HORAOBITO"], errors="coerce").dt.time
C:\Users\Kaue Mandarino\AppData\Local\Temp\ipykernel_7500\2821733182.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df["HORAOBITO"].astype(str), errors="coerce").dt.strftime("%H:%M:%S")


## 📊 Enriquecimento da Base com Faixa Horária (Dimensão de Tempo)

Este processo tem como objetivo **classificar o horário do óbito (`HORAOBITO`) em faixas e subfaixas horárias**, sem perder registros da base principal, seguindo um **modelo dimensional (fato + dimensão)** adequado para BI, analytics e modelagem analítica.

---

### 🎯 Objetivo
- Normalizar a coluna `HORAOBITO`
- Criar uma **coluna âncora ordinal (`ordem_subfaixa`)**
- Associar cada horário a uma **faixa horária** e **subfaixa horária**
- Realizar o *join* com a dimensão **sem perda de dados**

---

### 🧱 Estrutura da Dimensão `dim_faixa_horaria`

A dimensão representa a divisão lógica do dia em blocos de tempo:

| ordem_subfaixa | faixa_horaria | subfaixa_horaria |
|----------------|---------------|------------------|
| 1 | Madrugada | Madrugada Profunda |
| 2 | Madrugada | Madrugada Final |
| 3 | Manhã | Manhã Inicial |
| 4 | Manhã | Manhã Plena |
| 5 | Tarde | Horário de Almoço |
| 6 | Tarde | Tarde Inicial |
| 7 | Tarde | Tarde Final |
| 8 | Noite | Noite Inicial |
| 9 | Noite | Noite Profunda |

---

### 🔧 Etapas do Processo

#### 1️⃣ Normalização da Hora
A coluna `HORAOBITO` é convertida para o formato padrão `HH:MM:SS`, garantindo comparações consistentes e eliminando ruídos de formatação.

> Isso evita erros de inferência automática do Pandas e warnings de parsing.

---

#### 2️⃣ Criação da Coluna Âncora
A coluna `ordem_subfaixa` funciona como **chave técnica** que:
- Não depende de texto
- É estável
- Facilita joins, ordenações e métricas

Cada horário é mapeado para uma ordem fixa de subfaixa horária.

---

#### 3️⃣ Classificação por Intervalos
Os horários são classificados usando **intervalos explícitos**, garantindo:
- Leitura clara
- Regra determinística
- Facilidade de auditoria

Exemplo conceitual:
- `00:00–02:59` → Madrugada Profunda
- `11:30–13:29` → Horário de Almoço
- `21:00–23:59` → Noite Profunda

---

#### 4️⃣ Join Dimensional (Sem Perda de Dados)
O *merge* é feito com `how="left"`, garantindo que:
- Todos os registros da base principal permaneçam
- Dados não classificados fiquem como `NULL`, nunca excluídos

Esse padrão é essencial para **bases oficiais, dados de saúde e análises estatísticas**.

---

### 📈 Resultado Final
A base passa a conter:

- `HORAOBITO` (normalizada)
- `ordem_subfaixa` (chave âncora)
- `faixa_horaria` (camada 1)
- `subfaixa_horaria` (camada analítica)

In [47]:
# --- normaliza hora do óbito ---
df["HORAOBITO"] = (
    pd.to_datetime(
        df["HORAOBITO"]
        .astype(str)
        .str.extract(r"(\d{1,2}:\d{2}(:\d{2})?)")[0],
        format="%H:%M:%S",
        errors="coerce"
    )
    .dt.strftime("%H:%M:%S")
)

# --- cria coluna âncora por ordem ---
df["ordem_subfaixa"] = None

# --- aplica regras da faixa horária ---
df.loc[(df["HORAOBITO"] >= "00:00:00") & (df["HORAOBITO"] <= "02:59:59"), "ordem_subfaixa"] = 1
df.loc[(df["HORAOBITO"] >= "03:00:00") & (df["HORAOBITO"] <= "05:59:59"), "ordem_subfaixa"] = 2
df.loc[(df["HORAOBITO"] >= "06:00:00") & (df["HORAOBITO"] <= "08:59:59"), "ordem_subfaixa"] = 3
df.loc[(df["HORAOBITO"] >= "09:00:00") & (df["HORAOBITO"] <= "11:29:59"), "ordem_subfaixa"] = 4
df.loc[(df["HORAOBITO"] >= "11:30:00") & (df["HORAOBITO"] <= "13:29:59"), "ordem_subfaixa"] = 5
df.loc[(df["HORAOBITO"] >= "13:30:00") & (df["HORAOBITO"] <= "15:29:59"), "ordem_subfaixa"] = 6
df.loc[(df["HORAOBITO"] >= "15:30:00") & (df["HORAOBITO"] <= "17:59:59"), "ordem_subfaixa"] = 7
df.loc[(df["HORAOBITO"] >= "18:00:00") & (df["HORAOBITO"] <= "20:59:59"), "ordem_subfaixa"] = 8
df.loc[(df["HORAOBITO"] >= "21:00:00") & (df["HORAOBITO"] <= "23:59:59"), "ordem_subfaixa"] = 9

# --- tabela dimensão faixa horária ---
dim_faixa_horaria = pd.DataFrame({
    "ordem_subfaixa": [1,2,3,4,5,6,7,8,9],
    "faixa_horaria": ["Madrugada","Madrugada","Manhã","Manhã","Tarde","Tarde","Tarde","Noite","Noite"],
    "subfaixa_horaria": [
        "Madrugada Profunda",
        "Madrugada Final",
        "Manhã Inicial",
        "Manhã Plena",
        "Horário de Almoço",
        "Tarde Inicial",
        "Tarde Final",
        "Noite Inicial",
        "Noite Profunda"
    ]
})

# --- join sem perder registros ---
df = df.merge(dim_faixa_horaria, on="ordem_subfaixa", how="left")

# --- exibe resultado ---
df[["HORAOBITO", "ordem_subfaixa", "faixa_horaria", "subfaixa_horaria"]]


,HORAOBITO,ordem_subfaixa,faixa_horaria,subfaixa_horaria
0,20:24:00,8,Noite,Noite Inicial
1,NaN,None,NaN,NaN
2,NaN,None,NaN,NaN
3,NaN,None,NaN,NaN
4,NaN,None,NaN,NaN
...,...,...,...,...
16254,NaN,None,NaN,NaN
16255,20:00:00,8,Noite,Noite Inicial
16256,19:00:00,8,Noite,Noite Inicial
16257,15:40:00,7,Tarde,Tarde Final


In [48]:
f_estados = pd.DataFrame({
    "UF": ["AC","AL","AP","AM","BA","CE","DF","ES","GO","MA","MT","MS","MG","PA","PB",
           "PR","PE","PI","RJ","RN","RS","RO","RR","SC","SP","SE","TO"],
    "Estado": ["Acre","Alagoas","Amapá","Amazonas","Bahia","Ceará","Distrito Federal",
               "Espírito Santo","Goiás","Maranhão","Mato Grosso","Mato Grosso do Sul",
               "Minas Gerais","Pará","Paraíba","Paraná","Pernambuco","Piauí",
               "Rio de Janeiro","Rio Grande do Norte","Rio Grande do Sul","Rondônia",
               "Roraima","Santa Catarina","São Paulo","Sergipe","Tocantins"],
    "Região": ["Norte","Nordeste","Norte","Norte","Nordeste","Nordeste","Centro-Oeste",
               "Sudeste","Centro-Oeste","Nordeste","Centro-Oeste","Centro-Oeste",
               "Sudeste","Norte","Nordeste","Sul","Nordeste","Nordeste",
               "Sudeste","Nordeste","Sul","Norte","Norte","Sul","Sudeste","Nordeste","Norte"],
    "Habitantes (mi)": [0.83,3.1,0.73,4,14.1,9.2,3,4.1,7.2,6.9,3.8,2.8,20.5,8.1,4,
                        11.6,9.1,3.3,16.1,3.3,11.2,1.6,0.65,7.6,44.4,2.3,1.6],
    "Densidade": [5,112,5.6,2.7,24,56,522,76,21,21,4.2,7.8,36,6.5,66,
                  52,93,12,365,59,40,6.6,2.3,65,185,94,5.7],
    "PIB (R$ bi)": [17,66,18,116,364,194,286,190,251,124,233,142,857,215,80,
                     620,234,64,949,77,593,59,18,428,3000,54,51],
    "Renda per capita (R$)": [22000,20000,24000,27000,23000,24000,80000,46000,43000,
                               18000,55000,47000,41000,26000,22000,52000,25000,
                               21000,55000,26000,50000,32000,30000,58000,62000,
                               24000,28000]
})

In [49]:
df['NATURAL']

0        Rio Grande do Sul
1           Santa Catarina
2              Estrangeiro
3        Rio Grande do Sul
4        Rio Grande do Sul
               ...        
16254         Minas Gerais
16255               AmapÃ¡
16256             Amazonas
16257             Amazonas
16258             Amazonas
Name: NATURAL, Length: 16259, dtype: object

In [50]:
df["NATURAL"] = df["NATURAL"].replace({
    "AmapÃ¡": "Amapá",
    "ParaÃ­ba": "Paraíba",
    "PiauÃ­": "Piauí",
    "MaranhÃ£o": "Maranhão",
    "CearÃ¡": "Ceará",
    "EspÃ­rito Santo": "Espírito Santo",
    "SÃ£o Paulo": "São Paulo",
    "ParanÃ¡": "Paraná",
    "ParÃ¡": "Pará",
    "RondÃ´nia": "Rondônia",
    "RoraÃ­ma": "Roraima"
})

### Enriquecimento da coluna NATURAL com dados de Estados (f_estados)

Nesta etapa criamos uma **tabela de dimensão (`f_estados`)** contendo informações
socioeconômicas dos estados brasileiros (UF, região, população, PIB, renda, etc.).

Em seguida, corrigimos **pontualmente** alguns valores da coluna `NATURAL` que
estavam com problemas de encoding (ex.: `AmapÃ¡`, `ParÃ¡`, `SÃ£o Paulo`), garantindo
que o texto fique **idêntico** ao nome do estado na dimensão.  
Essa correção é feita de forma explícita, sem normalização automática, para manter
controle e legibilidade do dado.

Com os textos alinhados, realizamos um `merge` à esquerda entre o DataFrame
principal (`df`) e a dimensão `f_estados`, usando o nome do estado como chave.
As colunas da dimensão recebem o prefixo `f_estados_` para deixar claro que
são atributos derivados da tabela de estados.

Estados brasileiros passam a ter seus dados enriquecidos, enquanto valores como
`Estrangeiro` permanecem como `NaN`, o que é o comportamento correto para análise
e modelagem analítica
